In [1]:
!pip install pandas --q

In [2]:
import os
import requests
import pandas as pd
import sqlite3

In [3]:
# Download files

# URL
db_url = "https://datasets.imdbws.com/"

# File names from URL
files = [
    "name.basics.tsv.gz",
    "title.akas.tsv.gz",
    "title.basics.tsv.gz",
    "title.crew.tsv.gz",
    "title.episode.tsv.gz",
    "title.principals.tsv.gz",
    "title.ratings.tsv.gz"
]

# Local folder
data_path = "data"

# Check: path exists
os.makedirs(data_path, exist_ok=True)

# Download files
for file in files:
    url = db_url + file
    local_path = os.path.join(data_path, file)

    # If the file exists, then ignore, else download locally
    if not os.path.exists(local_path):
        print(f'Downloading {file}...')
        response = requests.get(url)

        # Request done? (status = 200)
        if response.status_code == 200:
            with open(local_path, 'wb') as f:
                f.write(response.content)
            print(f'{file} downloaded!')
        else:
            print(f'Download failed... Status code: {response.status_code}')
    else:
        print(f'{file} already exists.')

print('Download concluded!') 

name.basics.tsv.gz already exists.
title.akas.tsv.gz already exists.
title.basics.tsv.gz already exists.
title.crew.tsv.gz already exists.
title.episode.tsv.gz already exists.
title.principals.tsv.gz already exists.
title.ratings.tsv.gz already exists.
Download concluded!


In [ ]:
# Data transformation

# Directories
data_path = "data"
data_preprocessed_path = os.path.join(data_path, "preprocessed")

# Check: path exists
os.makedirs(data_preprocessed_path, exist_ok=True)

# Files
files = os.listdir(data_path)

# Open, preprocess and save data
for file in files:
    file_path = os.path.join(data_path, file)

    if os.path.isfile(file_path) and file.endswith('.gz'):
        print(f'Reading and preprocessing the file {file}...')

        # Read the file and transform to pandas
        df = pd.read_csv(file_path, sep='\t', compression='gzip', low_memory=False)

        # Replace "\\N" by NaN
        df.replace({'\\N': None}, inplace=True)

        # Save the DataFrame on preprocessed folder without compression
        destiny_path = os.path.join(data_preprocessed_path, file[:-3]) # remove .gz extension
        df.to_csv(destiny_path, sep='\t', index=False)

        print(f'Preprocess concluded for {file}! File saved on {data_preprocessed_path}')

print(f'All the files were preprocessed and downloaded on "preprocessed" directory!')

Reading and preprocessing the file title.principals.tsv.gz...
Preprocess concluded for title.principals.tsv.gz! File saved on data\preprocessed
Reading and preprocessing the file title.ratings.tsv.gz...
Preprocess concluded for title.ratings.tsv.gz! File saved on data\preprocessed
All the files were preprocessed and downloaded on "preprocessed" directory!


In [20]:
# Check DataFrame
df = pd.read_csv('data/preprocessed/name.basics.tsv', sep='\t')
df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0027125,tt0031983"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"


In [21]:
# Verify NaN
df.isna().sum()

nconst                      0
primaryName                59
birthYear            13637187
deathYear            14037145
primaryProfession     2798282
knownForTitles        1631323
dtype: int64

In [7]:
# Save on DB using SQLite

# Directories
preprocessed_path = os.path.join("data", "preprocessed")
db = "imdb_data.db"

# Connect to SQLite DB
conn = sqlite3.connect(db)

# List all the saved files
files = os.listdir(preprocessed_path)

# Read and save the files into DB
for file in files:
    file_path = os.path.join(preprocessed_path, file)

    if os.path.isfile(file_path) and file.endswith('.tsv'):
        # Read the TSV file using pandas
        df = pd.read_csv(file_path, sep='\t', low_memory=False)

        # Remove extension from the file name
        table_name = os.path.splitext(file)[0]

        # Replace special characters in the file name
        table_name = table_name.replace('.', '_').replace('-', '_')

        # Save the DataFrame into DB
        df.to_sql(table_name, conn, index=False, if_exists='replace')

        print(f'File {file} was saved as table {table_name} and uploaded into the SQLite DB.')

# Close the connection with DB
conn.close()

print('All the files were saved into DB!')

File name.basics.tsv was saved as table name_basics and uploaded into the SQLite DB.
File title.akas.tsv was saved as table title_akas and uploaded into the SQLite DB.
File title.basics.tsv was saved as table title_basics and uploaded into the SQLite DB.
File title.crew.tsv was saved as table title_crew and uploaded into the SQLite DB.
File title.episode.tsv was saved as table title_episode and uploaded into the SQLite DB.
File title.principals.tsv was saved as table title_principals and uploaded into the SQLite DB.
File title.ratings.tsv was saved as table title_ratings and uploaded into the SQLite DB.
All the files were saved into DB!


In [15]:
# DB name
db = "imdb_data.db"

# Connection to DB
conn = sqlite3.connect(db)

# Create a cursor
cursor = conn.cursor()

# Execute a query to get table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Get query results
tables = cursor.fetchall()

# Table names
print('DB table names:')
for tab in tables:
    print(tab[0])

# Close the cursor and connection
cursor.close()
conn.close()

DB table names:
name_basics
title_akas
title_basics
title_crew
title_episode
title_principals
title_ratings
analytics_titles
analytics_principals


In [11]:
# DB name
db = "imdb_data.db"

# Connection to DB
conn = sqlite3.connect(db)

# Create a cursor
cursor = conn.cursor()

# Execute query to get 10 first lines from name_basics (test)
query = """
WITH participants AS (
    SELECT
        tconst,
        COUNT(DISTINCT nconst) AS numParticipants
    FROM title_principals
    GROUP BY 1
),

languages AS (
    SELECT DISTINCT
        titleId AS tconst,
        language
    FROM title_akas
    WHERE language IS NOT NULL
)

SELECT DISTINCT
    tb.tconst,
    tb.titleType,
    tb.originalTitle,
    tb.startYear,
    tb.endYear,
    tb.genres,
    ta.language,
    tr.averageRating,
    tr.numVotes,
    tp.numParticipants
FROM title_basics AS tb
LEFT JOIN languages AS ta
    ON tb.tconst = ta.tconst
LEFT JOIN title_ratings AS tr
    ON tb.tconst = tr.tconst
LEFT JOIN participants AS tp
    ON tb.tconst = tp.tconst
LIMIT 10
;
"""

df = pd.read_sql_query(query, conn)

# Close DB connection
conn.close()

In [12]:
df.head()

,tconst,titleType,originalTitle,startYear,endYear,genres,language,averageRating,numVotes,numParticipants
0,tt0000001,short,Carmencita,1894.0,None,"Documentary,Short",ja,5.7,2142,3.0
1,tt0000002,short,Le clown et ses chiens,1892.0,None,"Animation,Short",ja,5.5,290,2.0
2,tt0000003,short,Pauvre Pierrot,1892.0,None,"Animation,Comedy,Romance",ja,6.4,2178,4.0
3,tt0000004,short,Un bon bock,1892.0,None,"Animation,Short",ja,5.3,186,2.0
4,tt0000005,short,Blacksmith Scene,1893.0,None,Short,en,6.2,2912,3.0


In [4]:
# DB name
db = "imdb_data.db"

# Connection to DB
conn = sqlite3.connect(db)

# Create a cursor
cursor = conn.cursor()

# Execute query to get 10 first lines from name_basics (test)
query = """
SELECT DISTINCT
    tp.nconst,
    tp.tconst,
    tp.ordering,
    tp.category,
    tb.genres
FROM title_principals AS tp
LEFT JOIN title_basics AS tb
    ON tp.tconst = tb.tconst
LIMIT 10
;
"""

df = pd.read_sql_query(query, conn)

# Close DB connection
conn.close()

In [5]:
df.head()

,nconst,tconst,ordering,category,genres
0,nm1588970,tt0000001,1,self,"Documentary,Short"
1,nm0005690,tt0000001,2,director,"Documentary,Short"
2,nm0005690,tt0000001,3,producer,"Documentary,Short"
3,nm0374658,tt0000001,4,cinematographer,"Documentary,Short"
4,nm0721526,tt0000002,1,director,"Animation,Short"


In [ ]:
analytics_titles = """
CREATE TABLE IF NOT EXISTS analytics_titles AS

WITH principals AS (
    SELECT
        tconst,
        COUNT(DISTINCT nconst) AS numParticipants
    FROM title_principals
    GROUP BY 1
),

languages AS (
    SELECT DISTINCT
        titleId AS tconst,
        language
    FROM title_akas
    WHERE language IS NOT NULL
)

SELECT DISTINCT
    tb.tconst,
    tb.titleType,
    tb.originalTitle,
    tb.startYear,
    tb.endYear,
    tb.genres,
    ta.language,
    tr.averageRating,
    tr.numVotes,
    tp.numParticipants
FROM title_basics AS tb
LEFT JOIN languages AS ta
    ON tb.tconst = ta.tconst
LEFT JOIN title_ratings AS tr
    ON tb.tconst = tr.tconst
LEFT JOIN principals AS tp
    ON tb.tconst = tp.tconst
;
"""

analytics_principals = """
CREATE TABLE IF NOT EXISTS analytics_principals AS

SELECT DISTINCT
    tp.nconst,
    tp.tconst,
    tp.ordering,
    tp.category,
    tb.genres
FROM title_principals AS tp
LEFT JOIN title_basics AS tb
    ON tp.tconst = tb.tconst
"""

In [14]:
# Lists of create table queries
create_table_queries = [analytics_titles, analytics_principals]

# DB name
db = 'imdb_data.db'

create_table_queries_indexes = zip(
    create_table_queries,
    range(1, len(create_table_queries)) # Index for print
    )

for query, i in create_table_queries_indexes:

    # Connection to DB
    conn = sqlite3.connect(db)

    # Cursor
    cursor = conn.cursor()

    # Execute query
    cursor.execute(query)

    # Close connection
    conn.close()

    print(f'Analytical table {i} was created succesfully.')

print('Tables were created succesfully!')

Tables were created succesfully!
